Writing a Basic Simple Program to Classify Dogs and Cats into separate classes using a basic Logistic Regression Based Neural network.

Using Most Basic Libraries and will write custom functions for the NN.

Libraries Used:

- Numpy : For computational operations. 
- Pandas : DataFrame Based Manipulations 
- cv2 : For reading the image data. Used to read/ convert image into array and generate grayscale.
- matplotlib : For plotting the images data. 

In [1]:
import pandas as pd;
import numpy as np;
import cv2;
import matplotlib.pyplot as plt;
import matplotlib;
import os;
from collections import Counter;
import random; 
%matplotlib inline

In [2]:
os.listdir('C:/Users/adity/Documents/Projects/')

['dogs-vs-cats.zip',
 'Experiments-with-Deep-Learning-',
 'sampleSubmission.csv',
 'test1',
 'test1.zip',
 'train',
 'train.zip']

In [3]:
#Reading the data 
# os.listdir("C:\\Users\\adity\\Documents\\Projects")

maindir="C:/Users/adity/Documents/Projects/"
trainDir="train"
path=os.path.join(maindir,trainDir);
X=[];
y=[];
for p in os.listdir(path):
    class_val=p.split(".")[0];
    img_array=cv2.imread(os.path.join(path,p),cv2.IMREAD_GRAYSCALE)
    new_img_array=cv2.resize(img_array,dsize=(80,80));
#     plt.imshow(new_img_array)
    X.append(new_img_array);
    y.append(class_val);
X=np.array(X).reshape(-1,80,80,1);
X_flatten=X.reshape(X.shape[0],-1).T
y_flatten=np.array(y)

# testDir="test1"
# path=os.path.join(maindir,testDir);
# X_test=[];
# y_test=[];
# #Reading the Test Data.
# for p in os.listdir(path):
#     class_val=p.split(".");
#     img_array=cv2.imread(os.path.join(path,p),cv.IMREAD_GRAYSCALE)
#     new_img_array=cv.resize(img_array,dsize=(80,80))
#     X_test.append(new_img_array);
#     y_test.append(class_val);
# X_test=np.array(X_test).reshape(-1,80,80,1)
# X_test_flatten=X_test.reshape(X_test.shape[0],-1).T
# y_test=np.array(y_test)
    

In [4]:
y_train=y_flatten.reshape(y_flatten.shape[0],1)
X_train=X_flatten.copy()

print('Shape of X in Training DataSet ',X_train.shape)
print('Shape of y in Training Dataset',y_train.shape)

Shape of X in Training DataSet  (6400, 25000)
Shape of y in Training Dataset (25000, 1)


In [5]:
#Standardizing the Dataset 


In [6]:
# np.unique(y_train,return_counts=True)

###  Image Data Preprocessing.

* Once we've read the data we'd want to preprocess and generate training and validation sets before proceeding to the test set and final results generation
* Also, class cat will be assigned 1 and dog 0. ( Although I love dogs over cats ). 

In [7]:
#Splitting the data into training and validation portions 
X_train=X_train/255.
data=pd.DataFrame(X_train).T
data.shape
data['Class']=y_train
data_cat=data[data['Class']=='cat'].reset_index()
data_dog=data[data['Class']=='dog'].reset_index()
indices=random.sample(range(12500),12500)
data_cat_train=data_cat.iloc[indices[:11250],:]
data_cat_validation=data_cat.iloc[indices[11250:],:]
data_dog_train=data_dog.iloc[indices[:11250],:]
data_dog_validation=data_dog.iloc[indices[11250:],:]

# data_cat_train.shape
data_train=pd.concat([data_cat_train,data_dog_train])
data_validation=pd.concat([data_cat_validation,data_dog_validation])

data_train=data_train.reset_index().iloc[:,1:]
data_train['NUM_CLASS']=0
data_train.loc[data_train['Class']=='cat','NUM_CLASS']=1
data_train=data_train.drop(['Class','index'],axis=1)
data_train=data_train.T
data_validation=data_validation.reset_index().iloc[:,1:]
data_validation['NUM_CLASS']=0
data_validation.loc[data_validation['Class']=='cat','NUM_CLASS']=1
data_validation=data_validation.drop(['Class','index'],axis=1)
data_validation=data_validation.T

In [8]:
print('Data Train Shape',data_train.shape)
print('Data Validation Shape',data_validation.shape)

Data Train Shape (6401, 22500)
Data Validation Shape (6401, 2500)


In [9]:
''' Generating sigmoid of a value '''
def sigmoid(z):
    s=1/(1+np.exp(-z))
    return s;

In [10]:
''' Parameter Initialization'''
def initialize_parameters(dim):
    w= np.zeros((dim,1));
    b=np.zeros(1);
    return w,b;

# t_w,t_b=initialize_parameters(2);
# print('w',t_w)
# print('b',t_b)

In [11]:
''' Generating forward and backward propagation'''
def propagate(w,b,X,Y):
    m=X.shape[1]
    #Forward Propagation
    A=sigmoid(np.dot(w.T,X)+b); #Computing activation.
    cost=(-1/m)*np.sum(np.multiply(Y,np.log(A))+ np.multiply((1-Y),np.log(1-A)))
    #Backward Propagation 
    dw= (1/m)*np.dot(X,(A-Y).T)
    db=1/m*(np.sum(A-Y))
    assert(dw.shape == w.shape)
    assert(db.dtype == float)
    cost = np.squeeze(cost)
    assert(cost.shape == ())
    
    grads = {"dw": dw,
             "db": db}
    
    return grads, cost

    

In [12]:
# w, b, X, Y = np.array([[1.],[2.]]), 2., np.array([[1.,2.,-1.],[3.,4.,-3.2]]), np.array([[1,0,1]])
# grads, cost = propagate(w, b, X, Y)
# print ("dw = " + str(grads["dw"]))
# print ("db = " + str(grads["db"]))
# print ("cost = " + str(cost))

In [15]:
''' Iteration and Optimization of the parameters'''
def optimize(w,b,X,Y,num_iterations,learning_rate,print_cost=False):
    costs=[];
    for i in range(num_iterations):
        grads,cost=propagate(w,b,X,Y); #calculating cost and gradients.
        dw = grads["dw"]
        db = grads["db"]

        w= w-( learning_rate*dw); #updating value of w.
        b=b -( learning_rate*db); # updating value of b
        
        if i%100==0:
            costs.append(cost);
        
        if print_cost and i%100==0:
            print('Iteration number %i: %f :'%(i,cost));
        
        params={'w': w,'b':b};
        grads={'dw':dw,'db':db}
        
    return params,grads,costs
    

In [16]:
# #Testing if the function actually works.
# params, grads, costs = optimize(w, b, X, Y, num_iterations= 100, learning_rate = 0.009, print_cost = False)
# print ("w = " + str(params["w"]))
# print ("b = " + str(params["b"]))
# print ("dw = " + str(grads["dw"]))
# print ("db = " + str(grads["db"]))

In [22]:
def predict(w,b,X):
    m=X.shape[1]; # Getting  number of examples since the shape of X is (number_of_features, number of examples)
    Y_prediction=np.zeros((1,m));
    w= w.reshape(X.shape[0],1);
    A=sigmoid(np.dot(w.T,X)+b);
    
    for i in range(A.shape[1]):
        Y_prediction[:,i]=np.where(A[:,i]>=0.5,1,0);
    assert (Y_prediction.shape==(1,m) )
    return Y_prediction;
    

In [23]:
# w = np.array([[0.1124579],[0.23106775]])
# b = -0.3
# X = np.array([[1.,-1.1,-3.2],[1.2,2.,0.1]])
# print ("predictions = " + str(predict(w, b, X)))

In [24]:
def model(X_train,Y_train,X_test,Y_test,num_iterations=2000,learning_rate=0.5,print_cost=False):
    
    #Initialize parameters
    w,b=initialize_parameters(X_train.shape[0]);
#     print('X_train shape',X_train.shape,Y_train.shape)
    
    #Gradient Descent 
    params,grads,costs=optimize(w,b,X_train,Y_train,num_iterations,learning_rate,print_cost);
    
    #Getting the parameters from the trained model.
    w=params['w']
    b=params['b']
    
    #Generating predictions for training and testing examples 
    Y_prediction_train=predict(w,b,X_train)
    Y_prediction_test=predict(w,b,X_test)
    
    #Print train/test Errors 
    print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))
    
    
    d={'costs':costs,'Y_prediction_train':Y_prediction_train,'Y_prediction_test':Y_prediction_test,'w':w,'b':b,'learning_rate':learning_rate,'num_iterations':num_iterations}
    return d;


In [25]:
train_set_x=data_train.iloc[:-1,:].to_numpy()
train_set_y=data_train.iloc[-1,:].to_numpy()
test_set_x=data_validation.iloc[:-1,:].to_numpy()
test_set_y=data_validation.iloc[-1,:].to_numpy()

In [26]:
d = model(train_set_x, train_set_y, test_set_x, test_set_y, num_iterations = 2000, learning_rate = 0.005, print_cost = True)

Iteration number 0: 0.693147 :
Iteration number 100: 0.681091 :
Iteration number 200: 0.678618 :
Iteration number 300: 0.677396 :
Iteration number 400: 0.676582 :
Iteration number 500: 0.675947 :
Iteration number 600: 0.675406 :
Iteration number 700: 0.674921 :
Iteration number 800: 0.674474 :
Iteration number 900: 0.674057 :
Iteration number 1000: 0.673661 :
Iteration number 1100: 0.673283 :
Iteration number 1200: 0.672920 :
Iteration number 1300: 0.672570 :
Iteration number 1400: 0.672232 :
Iteration number 1500: 0.671903 :
Iteration number 1600: 0.671583 :
Iteration number 1700: 0.671271 :
Iteration number 1800: 0.670966 :
Iteration number 1900: 0.670667 :
train accuracy: 59.56 %
test accuracy: 57.72 %
